In [1]:
import logging
# logging.basicConfig(level=logging.DEBUG)
from openmxr.song import Song

Output()

In [2]:


song: Song = Song.find(spotify_link="https://open.spotify.com/track/6l9Vr4mIGOH2MHnxyiqUGY")

[youtube] Extracting URL: https://music.youtube.com/watch?v=pOFvXlGAPc0

[youtube] pOFvXlGAPc0: Downloading webpage

[youtube] pOFvXlGAPc0: Downloading ios player API JSON

[youtube] pOFvXlGAPc0: Downloading android player API JSON

[youtube] pOFvXlGAPc0: Downloading ios music player API JSON

...

[youtube] pOFvXlGAPc0: Downloading web music player API JSON

[youtube] pOFvXlGAPc0: Downloading m3u8 information

[youtube] pOFvXlGAPc0: Downloading m3u8 information

  0%|          | 0.00/4.59M [00:00<?, ?B/s]

Compressing:   0%|          | 0.00/212M [00:00<?, ?B/s]

In [3]:
if not song.youtube_meta["heatmap"]:
    pass
items = sorted(song.youtube_meta["heatmap"], reverse=True, key=lambda x:x["value"])[:3]
heat_moments = [int(i["start_time"] * song.sample_rate+i["end_time"] * song.sample_rate)//2 for i in items]

In [4]:
heat_moments

[5895360, 5762880, 5630400]

In [11]:
from scipy.signal import resample

# y2 = resample(song.audio[0], (len(song.audio[0])//song.sample_rate)*44100)
y2 = song.resampled(44100, True)

In [6]:

from madmom.features.downbeats import DBNDownBeatTrackingProcessor, RNNDownBeatProcessor

downbeats_proc = DBNDownBeatTrackingProcessor(beats_per_bar=[4], fps=100)
activations = RNNDownBeatProcessor()

activations = activations(y2, sample_rate=44100, num_channels=1)
downbeats = downbeats_proc(activations)

downbeats

array([[1.0000e-02, 1.0000e+00],
       [3.7000e-01, 2.0000e+00],
       [7.5000e-01, 3.0000e+00],
       ...,
       [2.6933e+02, 3.0000e+00],
       [2.6970e+02, 4.0000e+00],
       [2.7009e+02, 1.0000e+00]])

In [10]:
import librosa
beats = librosa.beat.beat_track(y=song.audio_mono, sr=song.sample_rate)
bpm = beats[0]

bpm

160.71428571428572

In [8]:
import numpy as np


chromagram = librosa.feature.chroma_stft(y=song.audio_mono, sr=song.sample_rate)
mean_chroma = np.mean(chromagram, axis=1)
chroma_to_key = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
estimated_key_index = np.argmax(mean_chroma)
estimated_key = chroma_to_key[estimated_key_index]
estimated_key


C:\Users\piotr\AppData\Roaming\Python\Python311\site-packages\librosa\core\pitch.py:102: UserWarning: Trying to 
estimate tuning from empty frequency set.
  return pitch_tuning(

'C'

In [9]:
mapped = [min([int(i*song.sample_rate) for i in downbeats[:, 0]], key=lambda x:abs(x-hotspot)) for hotspot in heat_moments]
[a/song.sample_rate for a in mapped]

[123.0, 120.0, 117.37]